In [260]:
from gurobipy import *
import gurobipy as gb
import numpy as np

In [261]:
class EV: # Electric Vehicles
    def __init__(self, scalability):
        self.B0 = 0.35 # initial energy
        self.energy = 0.35 # initial state of energy
        if scalability is True: # For grid Network
            self.Bmax = 1.0 # battery capacity
        else: # For Braess networks
            self.Bmax = 0.5
        # Traffic parameters
        self.Va = 900 # m/min, free-flow speed
        self.Wa = 450 # m/min, backward speed
        self.Ka = 0.12 # vehicles/min, jam density for each link 𝑎
        self.Qa = 36 # vehicles * m / min^2, maximum flow capacity
        self.Ca = 0.04 # vehicles/min, critical density
        self.Da = 36 # vehicles/min, demand rate

        self.omega = 0.33 # amount of energy received when traversing oncharging link per time unit
        self.epsilon = 0.00018 # electricity consumption rate
    

class ICV: # Internal Combustion Vehicles
    def __init__(self, scalability):
        # Traffic parameters
        self.Va = 900 # m/min, free-flow speed
        self.Wa = 450 # m/min, backward speed
        self.Ka = 0.12 # vehicles/min, jam density for each link 𝑎
        self.Qa = 36 # vehicles * m / min^2, maximum flow capacity
        self.Ca = 0.04 # vehicles/min, critical density
        self.Da = 36 # vehicles/min, demand rate

In [262]:
# Chose some parameters:
scalability = False # True for Braess networks, True for grid networks
budget = 2800


WCL = gb.Model()
WCL.modelSense = gb.GRB.MAXIMIZE
WCL.setParam('OutputFlag', 0) # this is used to quite the xpress outputs (no printing)

M = ["EV", "ICV"] # class of vehicles

# The following lists are taken with respect to problem scalability
# seguendo le figure del paper in un file apposito scriviamo il dizionario
######### per ora Braess network
link_length = {1: 0, 2: 900, 3: 1800, 4: 900, 5: 3600, 6: 900, 7: 0}
#link = [i for i in range(len(link_length))] # lista dei link es. (1,2),(1,3)
link = list(link_length.keys())
link_source = link[0]
link_sink = link[-1]
length = list(link_length.values())
paths = [[1,2,4,6,7], [1,2,5,7], [1,3,6,7]] # lista dei percorsi
timestep = 0.25 # time steps che dipende da scalabilità
T = 60

In [263]:
x = WCL.addVars(link, vtype=gb.GRB.BINARY) # 1 if link a has WCL
y = WCL.addVars(M, len(paths), vtype=gb.GRB.BINARY) # 1 if path p is feasible for vehicle m
B = WCL.addVars(link, len(paths), lb=0.0, ub=0.5) # state of energy, bounded between being positive and max capacity

n = WCL.addVars(link, M, np.arange(0, T + timestep, timestep), lb=0, ub=432) # number of vehicles M on link 𝑎 at time t
u = WCL.addVars(link, M, np.arange(0, T + timestep, timestep), lb=0, ub=36) # incoming traffic flow of vehicle M to link 𝑎 at time t
v = WCL.addVars(link, M, np.arange(0, T + timestep, timestep), lb=0, ub=36) # outgoing traffic flow of vehicle M to link 𝑎 at time t
f = WCL.addVars(link, link, M, np.arange(0, T + timestep, timestep), lb=0, ub=36) # upstream traffic of vehicle M at link b, coming from downstream traffic at link a
# These variables are already defined with the constraint of non-negativity (formula 24)

n ha ub 2200 perché 36 veh/min * 60 min

In [264]:
len(paths[2])

4

In [265]:
# Feasibility of path
WCL.addConstr(gb.quicksum(length[a] * x[a] for a in link[1:-1]) <= budget) # formula 3: budget
# State of energy after travelling on link 𝑎 is no greater than the battery capacity
for p in range(len(paths)):
    # formula 4: initial state of energy
    a0 = paths[p][0]
    WCL.addConstr(B[a0,p] == 0.35)
    for i in range(1,len(paths[p])-1):
        b = paths[p][i-1]
        a = paths[p][i]
        print(f"i:",i,"b:",b,"a:",a)
        # formula 5: state of energy after traversing link 𝑎 on path 𝑝
        WCL.addConstr(B[a,p] <= B[b,p] -0.00018 * length[a-1] + 0.33 * length[a-1]/900 * x[a])
    for a in link[1:-1]: # formula 6: feasibility of path. M chosen to be 1000
        if a in paths[p]:
            WCL.addConstr(B[a,p] >= 1000 * (y["EV",p] - 1))
# Exactly one path must be chosen
WCL.addConstr(gb.quicksum(y["EV",p] for p in range(len(paths))) == 1)

i: 1 b: 1 a: 2
i: 2 b: 2 a: 4
i: 3 b: 4 a: 6
i: 1 b: 1 a: 2
i: 2 b: 2 a: 5
i: 1 b: 1 a: 3
i: 2 b: 3 a: 6


<gurobi.Constr *Awaiting Model Update*>

In [266]:
# Flow capacity

# Compute Kronecker's delta for constraint 19
deltaKron = {}
for p in range(len(paths)):
    for a in link:
        deltaKron[p,a] = 1 if a in paths[p] else 0

In [267]:
print(deltaKron)

{(0, 1): 1, (0, 2): 1, (0, 3): 0, (0, 4): 1, (0, 5): 0, (0, 6): 1, (0, 7): 1, (1, 1): 1, (1, 2): 1, (1, 3): 0, (1, 4): 0, (1, 5): 1, (1, 6): 0, (1, 7): 1, (2, 1): 1, (2, 2): 0, (2, 3): 1, (2, 4): 0, (2, 5): 0, (2, 6): 1, (2, 7): 1}


In [268]:
# alpha aggregate link-based share factor of vehicle class 𝑚 of formula 14, must be defined as a variable
# because it depends on u and otherwise it cannot be computed
alpha = WCL.addVars([link_source], M, np.arange(0, T + timestep, timestep), lb=0.0, ub=1.0)

# Compute alpha of formula 14: aggregate link-based share factor of vehicle class 𝑚
WCL.addConstr(gb.quicksum(alpha[link_source,m,t] for m in M for t in np.arange(0, T + timestep, timestep)) == 1)
alpha_EV = gb.quicksum(u[link_source,"EV",t] for t in np.arange(0, T + timestep, timestep))
alpha_ICV = gb.quicksum(u[link_source,"ICV",t] for t in np.arange(0, T + timestep, timestep))
den_alpha = alpha_EV + alpha_ICV
WCL.addConstr(gb.quicksum(alpha[link_source,"EV",t] for t in np.arange(0, T + timestep, timestep)) * den_alpha == alpha_EV)
WCL.addConstr(gb.quicksum(alpha[link_source,"ICV",t] for t in np.arange(0, T + timestep, timestep)) * den_alpha == alpha_ICV)

for t in np.arange(0, T + timestep, timestep):
    # formula 15: source link constraint is the demand rate of vehicle M at time step t
    WCL.addConstr(u[link_source,"EV",t] == 36)
    WCL.addConstr(u[link_source,"ICV",t] == 36)
    # formula 18: sink link constraint
    WCL.addConstr(v[link_sink,"EV",t] == 0)
    WCL.addConstr(v[link_sink,"ICV",t] == 0)

for a in link:
    for t in np.arange(0, T + timestep, timestep):
        # Formula 12: conservation of vehicle numbers
        WCL.addConstr(n[a,"EV",t] == gb.quicksum(u[a,"EV",k] - v[a,"EV",k] for k in np.arange(t+1)))
        WCL.addConstr(n[a,"ICV",t] == gb.quicksum(u[a,"ICV",k] - v[a,"ICV",k] for k in np.arange(t+1)))
        if a != link[0] and a!= link[-1]:
            # formula 13: upstream capacity
            t_in = max(0, round(t - length[a-1]/900))
            WCL.addConstr(gb.quicksum(u[a,"EV",k] for k in np.arange(t_in,t+1)) <= n[a,"EV",t])
            WCL.addConstr(gb.quicksum(u[a,"ICV",k] for k in np.arange(t_in,t+1)) <= n[a,"ICV",t])
            # formula 14: downstream capacity
            t_out = max(0, round(t - length[a-1]/450))
            WCL.addConstr(n[a,"EV",t] + gb.quicksum(v[a,"EV",k] for k in np.arange(t_out,t+1)) <= 0.12 * length[a-1] * alpha[link_source,"EV",t])
            WCL.addConstr(n[a,"ICV",t] + gb.quicksum(v[a,"ICV",k] for k in np.arange(t_out,t+1)) <= 0.12 * length[a-1] * alpha[link_source,"ICV",t])
            # formula 19: flow capacity of EV on links
            WCL.addConstr(gb.quicksum(f[b,a,"EV",t] for b in link) <= 36 * gb.quicksum(deltaKron[p,a] * y["EV",p] for p in range(len(paths))))
            WCL.addConstr(gb.quicksum(f[b,a,"ICV",t] for b in link) <= 36 * gb.quicksum(deltaKron[p,a] * y["ICV",p] for p in range(len(paths))))

        if a != link[0]: # no source link
            # formula 16: flux conservation for incoming vehicles
            WCL.addConstr(u[a,"EV",t] == gb.quicksum(f[b,a,"EV",t] for b in link))
            WCL.addConstr(u[a,"ICV",t] == gb.quicksum(f[b,a,"ICV",t] for b in link))
        if a != link[-1]: # no sink link
            # formula 17:  flux conservation for outgoing vehicles
            WCL.addConstr(v[a,"EV",t] == gb.quicksum(f[b,a,"EV",t] for b in link[:-1]))
            WCL.addConstr(v[a,"ICV",t] == gb.quicksum(f[b,a,"ICV",t] for b in link[:-1]))


## Altra modifica
Qua ho dovuto aggiungere S come variabile cui mettere un constraint (36 o l'espressione) perché non si può fare min(36, quicksum) perché Python non può confrontare un intero e una linear expression di Guroby

In [269]:
# Supply and demand at node

# Since the minimum between an int and a linear expression of Gurobi cannot be done,
# define supply and demand as variables with constraint to be smaller them
S = WCL.addVars(link, np.arange(0, T + timestep, timestep), lb=0)
D = WCL.addVars(link, np.arange(0, T + timestep, timestep), lb=0)
for a in link[1:-1]:
    for t in np.arange(0, T+timestep, timestep):
        ######### poi check se supply e demand sum sono uguali per EV e ICV e quindi basta calcolarle una volta
        # formula 20
        inflow_s = gb.quicksum(u[a,"EV",k] for k in np.arange(0,t))
        outflow_s = gb.quicksum(v[a,"EV",k] for k in np.arange(0, round(t - length[a-1]/900)))
        supply_sum = 0.12 * length[a-1] + outflow_s - inflow_s
        WCL.addConstr(S[a,t] <= 36)
        WCL.addConstr(S[a,t] <= supply_sum)
        # formula 22: 
        WCL.addConstr(u[a,"EV",t] <= S[a,t])
        
        # Idem for ICV
        inflow_s = gb.quicksum(u[a,"ICV",k] for k in np.arange(0,t))
        outflow_s = gb.quicksum(v[a,"ICV",k] for k in np.arange(0, round(t - length[a-1]/900)))
        supply_sum = 0.12 * length[a-1] + outflow_s - inflow_s
        WCL.addConstr(S[a,t] <= 36)
        WCL.addConstr(S[a,t] <= supply_sum)
        WCL.addConstr(u[a,"ICV",t] <= S[a,t])
                             
        # formula 21
        inflow_d = gb.quicksum(v[a,"EV",k] for k in np.arange(0,t))
        outflow_d = gb.quicksum(u[a,"EV",k] for k in np.arange(0, round(t - length[a-1]/450)))
        demand_sum = outflow_d - inflow_d
        WCL.addConstr(D[a,t] <= 36)
        WCL.addConstr(D[a,t] <= demand_sum)
        # formula 23: 
        WCL.addConstr(u[a,"EV",t] <= D[a,t])
        
        # Idem for ICV
        inflow_d = gb.quicksum(v[a,"ICV",k] for k in np.arange(0,t))
        outflow_d = gb.quicksum(u[a,"ICV",k] for k in np.arange(0, round(t - length[a-1]/450)))
        demand_sum = outflow_d - inflow_d
        WCL.addConstr(D[a,t] <= 36)
        WCL.addConstr(D[a,t] <= demand_sum)
        WCL.addConstr(u[a,"ICV",t] <= D[a,t])

In [270]:
WCL.setObjective(gb.quicksum((len(np.arange(0, T+timestep, timestep)) + 1 - t) * f[b, link[-1], m, t] for m in M for t in np.arange(0, T+timestep, timestep) for b in link[:-1]))
WCL.optimize()

In [271]:
if WCL.status == gb.GRB.OPTIMAL or WCL.status == gb.GRB.SUBOPTIMAL:
    for a in link:
        if x[a].x > 0.5:
            print(f"Link {a} ha WCL installato.")
else:
    print(f"⚠️ Il modello non è stato risolto correttamente. Status: {WCL.status}")

Link 5 ha WCL installato.
Link 6 ha WCL installato.


In [272]:
WCL.setParam("InfUnbdInfo", 1)
#WCL.computeIIS()
#WCL.write("model.ilp")

In [273]:
print("\n🔌 Link selezionati per WCL:")
for a in link:
    if x[a].x > 0.5:
        print(f"  - Link {a}")
        
print("\n🚗 Path selezionati per EV:")
for p in range(len(paths)):
    if y["EV", p].X > 0.5:
        print(f"  - Path {paths[p]}")
        
print("\n🔋 Stato di energia B[a, p]:")
for p in range(len(paths)):
    for a in link:
        if (a, p) in B:
            val = B[a, p].X
            if val > 1e-4:
                print(f"  B[{a}, {p}] = {val:.4f}")
                
print("\n📈 Flusso in entrata e uscita (solo se > 0):")
for a in link:
    for t in np.round(np.arange(0, T + timestep, timestep), 4):
        for m in M:
            if u[a, m, t].X > 1e-3 or v[a, m, t].X > 1e-3:
                print(f"  t={t:.2f}, link={a}, tipo={m}: u = {u[a,m,t].X:.2f}, v = {v[a,m,t].X:.2f}")
                
print("\n↔️ Flusso f[a, b, m, t] (solo se > 0):")
for a in link:
    for b in link:
        for m in M:
            for t in np.round(np.arange(0, T + timestep, timestep), 4):
                if f[a, b, m, t].X > 1e-3:
                    print(f"  f[{a},{b},{m},{t:.2f}] = {f[a,b,m,t].X:.2f}")
                    
print(f"\n🎯 Valore funzione obiettivo (outflow): {WCL.ObjVal:.2f}")


🔌 Link selezionati per WCL:
  - Link 5
  - Link 6

🚗 Path selezionati per EV:
  - Path [1, 2, 4, 6, 7]

🔋 Stato di energia B[a, p]:
  B[1, 0] = 0.3500
  B[2, 0] = 0.1620
  B[1, 1] = 0.3500
  B[1, 2] = 0.3500

📈 Flusso in entrata e uscita (solo se > 0):
  t=0.00, link=1, tipo=EV: u = 36.00, v = 0.00
  t=0.00, link=1, tipo=ICV: u = 36.00, v = 0.00
  t=0.25, link=1, tipo=EV: u = 36.00, v = 0.00
  t=0.25, link=1, tipo=ICV: u = 36.00, v = 0.00
  t=0.50, link=1, tipo=EV: u = 36.00, v = 0.00
  t=0.50, link=1, tipo=ICV: u = 36.00, v = 0.00
  t=0.75, link=1, tipo=EV: u = 36.00, v = 0.00
  t=0.75, link=1, tipo=ICV: u = 36.00, v = 0.00
  t=1.00, link=1, tipo=EV: u = 36.00, v = 0.00
  t=1.00, link=1, tipo=ICV: u = 36.00, v = 0.00
  t=1.25, link=1, tipo=EV: u = 36.00, v = 0.00
  t=1.25, link=1, tipo=ICV: u = 36.00, v = 0.00
  t=1.50, link=1, tipo=EV: u = 36.00, v = 0.00
  t=1.50, link=1, tipo=ICV: u = 36.00, v = 0.00
  t=1.75, link=1, tipo=EV: u = 36.00, v = 0.00
  t=1.75, link=1, tipo=ICV: u = 36